In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Observability

Given a linear system of the type

\begin{cases}
\dot{\textbf{x}}=A\textbf{x}+B\textbf{u} \\ 
\textbf{y}=C\textbf{x}
\end{cases}

we want to know if, by measuring the time history of the output $y(t)$ and of the input $u(t)$, it is possible to distinguish the initial state of the system that has generated the measured output. 
This kind of information about the system is related with the property called **Observability**.

First of all let us introduce the concept of <u>indistinguishability</u> for two states. 
>Two states $\bar{x_1}$ and $\hat{x_1}$ are indistinguishable in the future (here $\bar{x_1}I_{t_2}\hat{x_1}$) _if and only if_ $\forall u \in U \quad y(t,t_1,\bar{x_1},u)=y(t,t_1,\hat{x_1},u) \quad \forall t\in [t_1,t_2]$. Where $U$ is the set of the input values. 

In other words: two states are indistinguishable in the future if and only if the system, starting from them and evolving with the same input $u(t)$ applied, produce exactly the same output $y(t)$ in the time window considered. It can be shown that for continuous-time linear time invariant systems, indistinguishability (and observability) is independent of the time window.  

It is worth noting that the indistinguishability between two states (between $x_1$ and $x_2$) is the same as the indistinguishability for the difference of the two states with the state origin (between $x_1-x_2$ and $0$). 
Mathematically, equating the output response starting from the two initial conditions yields:

\begin{cases}
\bar{y}(t) = Ce^{At}\bar{x_1} + C\int_{t_1}^{t}e^{A(t-\tau)}Bu(\tau)d\tau \\ 
\hat{y}(t) = Ce^{At}\hat{x_1} + C\int_{t_1}^{t}e^{A(t-\tau)}Bu(\tau)d\tau
\end{cases}

$$
Ce^{At}\bar{x_1} = Ce^{At}\hat{x_1} \rightarrow Ce^{At}\underbrace{(\bar{x_1}-\hat{x_1})}_{\widetilde{x_1}} = Ce^{At}0 \,.
$$

This implies that we can talk about indistinguishability from the origin, and if a state is undistinguishable from the origin it is said to be unobservable. 

A measure of observability of a system can be introduced using the _observability Graminan_ $G_o(0,t_f)=\int_{0}^{t_f}e^{A^T\tau}C^TCe^{A\tau}d\tau$. It can be demonstrated that the space of the non observable states of a system is the nullspace of $G_o$; so, a linear system is observable if and only if $\text{ker}(G_o)=\{0\}$ or, equivalently if it's true that $\widetilde{x}I_{t}0 \Rightarrow \widetilde{x}=\{0\}$.

Defining the observability matrix as 
$$
\mathcal{O} = \begin{bmatrix}C \\ CA \\ CA^2 \\ \vdots \\ CA^{n-1} \end{bmatrix}
$$
it can be demonstrated that $\text{ker}(G_o)=\text{ker}(\mathcal{O})$ so,  the observability of a system can be investigated by studying the rank of the observability matrix i.e. if $\text{rank}(\mathcal{O})=n$ where $n: \, x\in \mathbb{R}^n$ the system is said to be observable.

Knowledge of observability of a system and of single states is fundamental beacuse it impacts which modes of the system appear on the output and which do not. If a system is Observable, all its modes can appear on the putput depending on initial conditions and input functions. If a system is not observable, that is if $\mathcal{O}$ is not full rank, then some modes of the system cannot be seen on the output. 

**NOTE:** A concept analogous to observability is _reconstructability_. This derives from the request to be able to reconstruct the final state in which the system is located instead of the initial state. For continuous-time linear systems, a system is reconstructable if and only if it is observable, in discrete time this property no longer holds. This can be easily proven by realizing that once the initial state has been _observed_, than the current state can be _recosntructed_ using the Lagrange Equation (since the input $u$ is known). Conversely, if the current state is known, the initial state can be obtained by integrating back in time.  

### How to use this notebook?

Presented below is an interactive example in which the matrices $A$ and $C$ can be edited.
- Try to make unobservable, if possible, only the third state $x_1$ of the default couple $(A,C)$ by changing only $C$.
- Try to make unobservable, if possible, the state $x_2$ of the default couple $(A,C)$ by changing only $C$.
- Try to make the couple $(A,C)$ unobservable by changing only $C$ and leaving the default $A$. Once done, think at the structure of $A$ and answer: Why it became unobservable? For what/which state(s) it the observability lost?
- With $C=[1, 1, 1]$ change $A$ in order to make $(A,C)$ unobservable (help yourself by thinking about the $\mathcal{O}$ matrix).
- Try to write a couple $(A,C)$ that has one positive eigenvalue that is associated to an unobservable state. How does the output behaves in the free response?

In [2]:
#Preparatory Cell 

%matplotlib inline
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'$\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}$']
     return ''.join(rv) #'\n'.join(rv)

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell
A = numpy.matrix('-1 1 0; 0 -1 1; 0 0 -1')
B = numpy.matrix('0; 0; 1')
C = numpy.matrix('1 0 0')
X0 = numpy.matrix('2; 2; 2')

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)

In [4]:
# Misc
#define type of ipout 
selu = widgets.Dropdown(
    options=['impulse', 'step', 'sinusoid', 'square wave'],
    value='step',
    description='Type of input:',
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=0,
    min=0,
    max=20.0,
    step=0.1,
    description=r'$u$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=1,
    min=0.01,
    max=4,
    step=0.01,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

In [7]:
def main_callback(A, B, C, X0, u, period, selu, DW):
    eig = numpy.linalg.eig(A)[0]
    O = control.obsv(A,C)
    rankO = numpy.linalg.matrix_rank(O)
    
    text = r'The eigenvalues of the system are ' + str(eig[0]) + r', ' + str(eig[1])+ r' and ' + str(eig[2]) + r'. '
    if rankO == 3:
        text = text + r'Since $\text{rank}(\mathcal{O})=3$ the system is observable. ' + '\n\n' + 'The observability matrix is: ' + r'$\mathcal{O}=$' + bmatrix(O) #+ r'$'
    else:
        text = text + r'Since $\text{rank}(\mathcal{O})=' + str(rankO) + r'$ the system is unobservable. ' + '\n\n' + 'The observability matrix is: ' + r'$\mathcal{O}=' + bmatrix(O) + r'$'
    display(Markdown(text))
    
    
    sys = sss(A,B,C,0)
    if numpy.real([eig[0],eig[1],eig[2]])[0] == 0 and numpy.real([eig[0],eig[1],eig[2]])[1] == 0 and numpy.real([eig[0],eig[1],eig[2]])[2] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))),1000)
    
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    
    fig = plt.figure(figsize=(16,5))
    
    fig.add_subplot(121)
    plt.title('Initial response: states')
    plt.plot(T,X[0])
    plt.plot(T,X[1])
    plt.plot(T,X[2])
    plt.ylabel('States')
    plt.xlabel('time [s]')
    plt.legend([r'$x_1$',r'$x_2$',r'$x_3$'])
    plt.grid()
    
    fig.add_subplot(122)
    plt.ylabel(r'$y$')
    plt.plot(T,Y)
    plt.xlabel('time [s]')
    plt.title('Initial response: output')
    plt.grid()
    
    
alltogether = widgets.VBox([widgets.HBox([widgets.Label('A:',border=3),  Aw, widgets.Label('   ',border=3),
                                          widgets.Label('B:',border=3),  Bw, widgets.Label('   ',border=3),
                                          widgets.Label('x0:',border=3),X0w,widgets.Label('   ',border=3),
                                          START]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([widgets.Label('C:',border=3),Cw,widgets.Label('   ',border=3)]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([selu, u, period])])
out = widgets.interactive_output(main_callback, {'A':Aw, 'B':Bw, 'C':Cw, 'X0':X0w, 'u':u, 'period':period, 'selu':selu, 'DW':DW})
out.layout.height = '460px'
display(out, alltogether)

Output(layout=Layout(height='460px'))

In [6]:
%%js
Jupyter.notebook.execute_cells([5])

<IPython.core.display.Javascript object>